In [ ]:
// File: src/utils/httpRequest.js
import axios from "axios";

const httpRequest = axios.create({
    baseURL: "https://api01.f8team.dev/api",
    timeout: 10000, // Nên set timeout
});

// ==========================================================
// 1. CẤU HÌNH & BIẾN TRẠNG THÁI
// ==========================================================
let isRefreshing = false;
// Hàng đợi lưu các request bị lỗi (Promise) để chờ token mới
let failedQueue = [];

/**
 * Hàm thêm request vào hàng đợi
 * @param {Function} resolve - Hàm resolve của Promise
 * @param {Function} reject - Hàm reject của Promise
 */
const addToQueue = (resolve, reject) => {
    failedQueue.push({ resolve, reject });
};

/**
 * Hàm xả hàng đợi sau khi refresh xong (thành công hoặc thất bại)
 * @param {Error} error - Lỗi nếu refresh thất bại
 * @param {String} token - Token mới nếu refresh thành công
 */
const processQueue = (error, token = null) => {
    failedQueue.forEach((prom) => {
        if (error) {
            prom.reject(error);
        } else {
            prom.resolve(token); // Truyền token mới về cho các request đang chờ
        }
    });
    failedQueue = []; // Xóa sạch hàng đợi
};

// ==========================================================
// 2. LOGIC GỌI API REFRESH TOKEN
// ==========================================================
const refreshAccessToken = async () => {
    try {
        const refreshToken = localStorage.getItem("refreshToken");
        // Gọi api refresh (lưu ý không dùng instance httpRequest để tránh loop interceptor)
        const response = await axios.post("https://api01.f8team.dev/api/auth/refresh-token", {
            refresh_token: refreshToken,
        });

        const { access_token, refresh_token } = response.data;
        
        // Lưu token mới
        localStorage.setItem("accessToken", access_token);
        localStorage.setItem("refreshToken", refresh_token);

        return access_token;
    } catch (error) {
        // Nếu refresh token cũng hết hạn hoặc lỗi -> Xóa storage & Reject
        localStorage.removeItem("accessToken");
        localStorage.removeItem("refreshToken");
        throw error;
    }
};

// ==========================================================
// 3. INTERCEPTOR (TRÁI TIM CỦA LOGIC)
// ==========================================================
httpRequest.interceptors.request.use(
    (config) => {
        // Luôn lấy token mới nhất từ storage trước khi gửi request đi
        const token = localStorage.getItem("accessToken");
        if (token) {
            config.headers.Authorization = `Bearer ${token}`;
        }
        return config;
    },
    (error) => Promise.reject(error)
);

httpRequest.interceptors.response.use(
    (response) => response.data, // Trả về data luôn cho gọn
    async (error) => {
        const originalRequest = error.config;

        // Nếu lỗi không phải 401 hoặc request này đã từng retry rồi -> Reject luôn
        if (error.response?.status !== 401 || originalRequest._retry) {
            return Promise.reject(error);
        }

        // --- [QUAN TRỌNG] TỐI ƯU HÓA: XỬ LÝ REQUEST THỨ 11 (Double Refresh) ---
        // Kiểm tra xem Token trong LocalStorage có khác với Token trong header của request lỗi không.
        // Nếu khác nhau => Có nghĩa là đã có ai đó refresh xong rồi.
        const currentToken = localStorage.getItem("accessToken");
        // Lấy token từ header Authorization (dạng "Bearer xyz...")
        const requestToken = originalRequest.headers['Authorization']?.split(' ')[1];

        if (currentToken && requestToken !== currentToken) {
            // Token đã được làm mới rồi! Không cần refresh nữa.
            // Cập nhật token mới vào request cũ và gọi lại ngay.
            originalRequest.headers['Authorization'] = `Bearer ${currentToken}`;
            return httpRequest(originalRequest);
        }
        
        // Đánh dấu request này đang retry để tránh lặp vô tận nếu vẫn lỗi
        originalRequest._retry = true;

        // --- TRƯỜNG HỢP 1: ĐANG CÓ TIẾN TRÌNH REFRESH (Followers) ---
        if (isRefreshing) {
            return new Promise((resolve, reject) => {
                addToQueue(resolve, reject);
            })
            .then((token) => {
                // Nhận token mới từ processQueue
                originalRequest.headers['Authorization'] = `Bearer ${token}`;
                return httpRequest(originalRequest);
            })
            .catch((err) => Promise.reject(err));
        }

        // --- TRƯỜNG HỢP 2: LÀ REQUEST ĐẦU TIÊN BỊ LỖI (Leader) ---
        isRefreshing = true;

        try {
            const newToken = await refreshAccessToken();
            
            // Refresh thành công -> Xả hàng đợi
            processQueue(null, newToken);

            // Gọi lại request của chính ông Leader
            originalRequest.headers['Authorization'] = `Bearer ${newToken}`;
            return httpRequest(originalRequest);

        } catch (refreshError) {
            // Refresh thất bại -> Báo lỗi cho toàn bộ hàng đợi
            processQueue(refreshError, null);
            
            // Xử lý logout hoặc điều hướng về login
            // window.location.href = '/login'; 
            return Promise.reject(refreshError);
        } finally {
            // Luôn tắt cờ refresh
            isRefreshing = false;
        }
    }
);

export default httpRequest;